In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
!pip install gdown
!gdown 1hKQDX7SZw5AzysFh32gr0qxTOHLzAQIC

Downloading...
From: https://drive.google.com/uc?id=1hKQDX7SZw5AzysFh32gr0qxTOHLzAQIC
To: /kaggle/working/SemEval2025_task11_processed.zip
100%|██████████████████████████████████████| 2.99M/2.99M [00:00<00:00, 92.5MB/s]


In [3]:
!rm -rf processed_data
!unzip -q SemEval2025_task11_processed.zip

In [4]:
!pip install -q transformers[torch] accelerate -U
!pip install -q datasets
!pip install -q transformers
!pip install -q iterative-stratification

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 100.5 MB/s eta 0:00:0000:010:01


In [5]:
import numpy as np
import pandas as pd
import os
from os import path
from tqdm.notebook import tqdm
from tqdm import tqdm
import string
import copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy
from datasets import Dataset,load_dataset, DatasetDict
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import classification_report,accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from scipy.special import expit


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
models_list_1=[
             "funnel-transformer/medium",
             "google/electra-base-discriminator",
             "facebook/bart-base",
        ]

models_name_1=[
"funnel-transformer-medium",
'electra-base',
"bart-base",
]

models_list_2=[
             "facebook/bart-large",
              "FacebookAI/xlm-roberta-base",
          ]

models_name_2=[
"bart-large",
"XLM-Roberta-base",
]

models_list_3=[
      "FacebookAI/xlm-roberta-large",
     "google-bert/bert-base-multilingual-uncased",
          ]

models_name_3=[
"XLM-Roberta-large",
"bert",
]

models_list_4=[

     "distilbert/distilroberta-base",
     "distilbert/distilbert-base-uncased",
     "distilbert/distilbert-base-multilingual-cased",
      ]


models_name_4=[
"distil-roberta-base",
"distil-bert",
"distil-bert multilingual",
]

models_list_5=[
     "microsoft/mdeberta-v3-base",
     "albert/albert-base-v2",
     "xlnet/xlnet-base-cased",
     "google-t5/t5-base",
      ]
models_name_5=[
"DeBERTAV3",
"albert-base",
"xlnet-base",
"t5",
]

In [8]:
kind="train"
task="track_a"
langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"

# train_data=pd.read_csv(processed_path)


In [9]:
all_csv=pd.DataFrame()
for lang in langs:
    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    if not os.path.isfile(processed_path):
      print(processed_path)
      continue
    train_data=pd.read_csv(processed_path)
    train_data.columns = train_data.columns.str.lower()
    all_csv = pd.concat([all_csv, train_data],ignore_index=True)



processed_data/train/track_a/sum.csv


In [10]:
train_data=all_csv.drop(columns={"unnamed: 0"})
train_data=train_data.fillna(0)
train_data

,id,anger,disgust,fear,joy,sadness,clean_message,surprise
0,afr_train_track_a_00001,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,afr_train_track_a_00002,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,afr_train_track_a_00003,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,afr_train_track_a_00004,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,afr_train_track_a_00005,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...
25557,tir_train_track_a_03677,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,tir_train_track_a_03678,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,tir_train_track_a_03679,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,tir_train_track_a_03680,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [11]:
train_data = train_data[
    train_data['clean_message'].isnull()==False]
train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,clean_message,surprise
0,0,afr_train_track_a_00001,0,0.0,0,1,0,ons sal gesondheidsprogramme in skole herinstel,0.0
1,1,afr_train_track_a_00002,0,0.0,0,1,0,ons moet verstaan dat ons in landbou een groot...,0.0
2,2,afr_train_track_a_00003,0,0.0,0,1,0,hulle is die trots van die boland en hulle hoo...,0.0
3,3,afr_train_track_a_00004,0,0.0,0,0,0,ek sou nog baie voorbeelde vir u kon noem,0.0
4,4,afr_train_track_a_00005,0,0.0,0,1,0,in ons bedryf loon dit gewoonlik om geduldig t...,0.0
...,...,...,...,...,...,...,...,...,...
25557,25557,tir_train_track_a_03677,0,0.0,0,0,1,ሕማቅ ስማዕ ጽንዓት ይሃብ ቤተስብን ምቅርብ መቃልስቱን መንግስተ ሰማይ የ...,0.0
25558,25558,tir_train_track_a_03678,1,0.0,0,0,0,ኣጃንዳ ክህቡና ደልዮም ዮም ዞም ሌያቡ,0.0
25559,25559,tir_train_track_a_03679,0,0.0,0,0,0,ኣብ መንጎ ኤርትራን ኢትዮጵያን ተፈጥሩ ዘሎ ሰላም ዝንቅንቆ የለን erit...,0.0
25560,25560,tir_train_track_a_03680,0,0.0,0,0,0,ብሪጣንያ ካብ ፈረንሳይ ዝሰግሩ ስደተኛታት ንምሓዝ ድሮንን ራዳርን ክጥቀም...,0.0


In [12]:
train_data=train_data.sample(frac=1)

In [13]:
#update label column name
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']

y = train_data[label_columns].values

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=1)
train_indices, valid_indices = next(mskf.split(train_data, y))

train = train_data.iloc[train_indices].reset_index(drop=True)
valid = train_data.iloc[valid_indices].reset_index(drop=True)

x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()
x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

y_train_df = pd.DataFrame(y_train, columns=label_columns)

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)
label_distribution.columns = ['count_0', 'count_1']
print(label_distribution)

          count_0  count_1
anger       16358     4092
fear        17984     2466
joy         16023     4427
sadness     16960     3490
surprise    18594     1856


In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [15]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):
  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)
  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
  train_dataset = Dataset(train_encodings, y_train)
  valid_dataset = Dataset(valid_encodings, y_valid)
  return train_dataset,valid_dataset

def train_arg():
    training_args = TrainingArguments(
    output_dir="./output/best-model-v1",
    report_to=None,
    num_train_epochs=3,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy=IntervalStrategy.STEPS,
    save_total_limit=1,
    logging_dir="./logs",
    logging_strategy=IntervalStrategy.STEPS,
    logging_steps=250,
    eval_steps=500,
    load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    warmup_steps=500,
  )
    return training_args

In [16]:
def compute_metrics(p):
  predicted_logits = p.predictions
  print(f"Type of predicted_logits: {type(predicted_logits)}")
  if isinstance(predicted_logits, tuple):
        predicted_logits = predicted_logits[0]
  if not isinstance(predicted_logits, np.ndarray):
        predicted_logits = np.array(predicted_logits)
  if len(predicted_logits.shape) == 3:
        predicted_logits = predicted_logits[:, 0, :]
  predicted_probabilities = expit(predicted_logits)
  threshold = 0.5
  predicted_classes = (predicted_probabilities >= threshold).astype(int)
  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average="weighted")
  accuracy = accuracy_score(y_valid, predicted_classes)
  return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

In [17]:
def evaluate_func(trainer):
  trainer.evaluate()
  predictions = trainer.predict(valid_dataset)
  predicted_logits = predictions.predictions
  print(f"Type of predicted_logits: {type(predicted_logits)}")
  if isinstance(predicted_logits, tuple):
        predicted_logits = predicted_logits[0]
  if not isinstance(predicted_logits, np.ndarray):
        predicted_logits = np.array(predicted_logits)
  if len(predicted_logits.shape) == 3:
        predicted_logits = predicted_logits[:, 0, :]
  predicted_probabilities = expit(predicted_logits)
  threshold = 0.5
  predicted_classes = (predicted_probabilities >= threshold).astype(int)
  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average=None)
  accuracy = accuracy_score(y_valid, predicted_classes)
  return precision, recall, f1,accuracy

In [18]:
MAX_LENGTH = 256
metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])
confusion_matrices = []
n_splits = 5
mskf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)

In [ ]:
models_list=models_list_3
models_name=models_name_3

# models_list=models_list_2
# models_name=models_name_2
print(models_list)
for x in range(len(models_list)):
  torch.cuda.empty_cache()

  i=models_list[x]
  print(i)
  tokenizer = AutoTokenizer.from_pretrained(i)
  model = AutoModelForSequenceClassification.from_pretrained(i, num_labels=5, ignore_mismatched_sizes=True)
  device="cuda" if torch.cuda.is_available() else 'cpu'
  model = model.to(device)
  for param in model.parameters():
      param.data = param.data.contiguous()

  training_args = train_arg()
  train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH)
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=valid_dataset,
      compute_metrics=compute_metrics
  )
  trainer.train()
  precision, recall, f1,accuracy=evaluate_func(trainer)
  new_data = pd.DataFrame({
    "Models_name": [models_name[x]],
    "Precision": [precision],
    "Recall": [recall],
    "F1_score": [f1],
    "Accuracy": [accuracy]
  })

  metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)

  model_confusion_matrices = []
  predicted_classes = trainer.predict(valid_dataset).predictions



  if isinstance(predicted_classes, tuple) :
    predicted_classes = predicted_classes[0]
  predicted_classes = np.array(predicted_classes)
  # print(f"Type after conversion: {type(predicted_classes)}")

  if len(predicted_classes.shape) > 2:
    predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)
    # print(f"Shape after reshaping: {predicted_classes.shape}")




  predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)

  y_valid_np = np.array(y_valid)

  for j in range(len(label_columns)):
      print(f"Computing confusion matrix for label: {label_columns[j]}")
      cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])

      # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])
      model_confusion_matrices.append(cm)
  confusion_matrices.append(model_confusion_matrices)


  print(f"{models_name[x]} evaluates:")
  for k, label in enumerate(label_columns):
    print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")
  print(f"Overall accuracy: {accuracy:.4f}")
  print("-------------------------------------------------------------------------\n")

['FacebookAI/xlm-roberta-large', 'google-bert/bert-base-multilingual-uncased']
FacebookAI/xlm-roberta-large


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Type of predicted_logits: <class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>
Type of predicted_logits: <class 'numpy.ndarray'>


Type of predicted_logits: <class 'numpy.ndarray'>


In [ ]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()
    if(len(lang_valid_data)==0):
      continue
    # Tokenize the validation data
    lang_valid_encodings = tokenizer(lang_x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    lang_valid_dataset = Dataset(lang_valid_encodings, lang_y_valid)

    # Get predictions
    predictions = trainer.predict(lang_valid_dataset,)
    predicted_logits = predictions.predictions

    if isinstance(predicted_logits, tuple):
        predicted_logits = predicted_logits[0]

    if not isinstance(predicted_logits, np.ndarray):
        predicted_logits = np.array(predicted_logits)

    if len(predicted_logits.shape) == 3:
        predicted_logits = predicted_logits[:, 0, :]

    predicted_probabilities = expit(predicted_logits)
    threshold = 0.5
    predicted_classes = (predicted_probabilities >= threshold).astype(int)

    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(lang_y_valid, predicted_classes, average="weighted")
    accuracy = accuracy_score(lang_y_valid, predicted_classes)

    # Store the metrics in the DataFrame
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
        "Accuracy": [accuracy]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)